<a href="https://colab.research.google.com/github/haidaraf6/MLCAPSTONE/blob/main/Copy_of_MODEL_CAPSTONE_ML_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import zipfile
import tensorflow_datasets as tfds


# Prepare the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/Colab/dataset.zip'

In [ ]:
!unrar x -Y "/content/drive/MyDrive/Colab/Rice Dataset.rar" "/content/dataset/"

In [ ]:
import zipfile

file_dataset = '/content/drive/MyDrive/Colab/dataset_capstone.zip'
zipdata = zipfile.ZipFile(file_dataset, 'r')
zipdata.extractall('dataset')
zipdata.close()


# Model Build

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, PReLU, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(22, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 256)       295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 256)      0

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Data Augmentation
*The settings can be changed later*


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = '/content/dataset/DATASET/Train'
training_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

VALIDATION_DIR = '/content/dataset/DATASET/Val'
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(200, 200),
    class_mode='categorical',
    batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(200, 200),
    class_mode='categorical',
    batch_size=32
)


Found 28347 images belonging to 22 classes.
Found 7261 images belonging to 22 classes.


In [ ]:
from google.colab import files

model.save('model.h5')
files.download('model.h5')

In [ ]:
model_h5 = '/content/drive/MyDrive/Colab/Model/my_model_rice_new.h5'
model=tf.keras.models.load_model(model_h5)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint=ModelCheckpoint("model.h5",
                           monitor='val_accuracy',
                           verbose=1,
                           save_base_only=True,
                           mode='max')


In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
      if logs['val_accuracy'] > 0.90:
        if logs['val_loss'] < 0.50:
          self.model.stop_training = True

myCallback = myCallback()

# Train the Model

In [ ]:
history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[myCallback, checkpoint])

Epoch 1/100
606/886 [===================>..........] - ETA: 41:29 - loss: 2.4055 - accuracy: 0.1622

Tes Akurasi dan Loss

In [ ]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Run code below to show accuracy plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.arange(20), history.history['accuracy'])
plt.plot(np.arange(20), history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()


# Predict image using the model

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
  # Memprediksi gambar
  path = fn
  img = load_img(path, target_size=(200, 200))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=100)
  
  # Mendapatkan indeks kelas dengan probabilitas tertinggi
  predicted_class_index = np.argmax(classes, axis=1)[0]
  
  # Daftar nama kelas yang sesuai dengan indeks kelas
  class_names = ["Rice Bacterial Leaf Blight", "Rice Brown Spot", "Rice Healthy", "Rice leaf blast", "Rice Leaf Scald", "Rice Narrow Brown Spot"] 

  # Mendapatkan string klasifikasi berdasarkan indeks kelas
  predicted_class = class_names[predicted_class_index]
  print(classes)
  print(fn)
  print("Hasil klasifikasi:", predicted_class)


In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
img = load_img('/content/rice3-1.jpg', target_size=(200, 200))
img = np.expand_dims(img, axis=0)
#images = np.vstack([img])
model.predict(img)

# Save the Trained Model

In [ ]:
#Save the trained model as a Keras HDF5 file. 
saved_model_path = "./my_model.h5"
model.save(saved_model_path)

In [ ]:
!cp '/content/my_model_rice90.h5' '/content/drive/MyDrive/Colab/Model'

In [ ]:
!cp '/content/model_rice_new.json' '/content/drive/MyDrive/Colab/Model'

Save Model to Json

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

Save to TensorFlow Lite

In [ ]:
model=tf.keras.models.load_model("model.h5")

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert

In [ ]:
with open ("model.tflite","wb") as tflite_file:
     tflite_file.write(converter)

In [ ]:
import os
from PIL import Image

folder_path = '/content/dataset/DATASET/Val'

def check_images(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                img.verify()  # Verifikasi gambar
            except (IOError, SyntaxError) as e:
                print("File rusak:", file_path)

check_images(folder_path)
